# Crisis vs. non-crisis analysis (distance method)

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, date, timedelta
from pandas.tseries.offsets import *
from pandas.tseries.holiday import USFederalHolidayCalendar

In [2]:
# Reading table present on the file about bear markets in history

bear_markets = pd.read_csv("../crisis_analysis/bear_markets.csv")
bear_markets = bear_markets.drop(columns=['return_percentage','days_duration'])

type_mapping = {
    'start': 'datetime64[ns]',
    'end': 'datetime64[ns]'
}

bear_markets = bear_markets.astype(type_mapping) #ensuring dtype is datetime
bear_markets.head(5)

,start,end
0,1929-09-07,1929-11-13
1,1930-04-10,1930-12-16
2,1931-02-24,1931-06-02
3,1931-06-27,1931-10-05
4,1931-11-09,1932-06-01


In [3]:
def create_daily_bear_market_indicator(bear_markets):
    '''
    Transform a dataframe of bear market start and end dates into a daily time series
    with an indicator for whether each date is in a bear market.
    
    Parameters:
    -----------
    bear_periods_df : pandas.DataFrame
        DataFrame with 'start' and 'end' columns containing dates of bear market periods
    
    Returns:
    --------
    pandas.DataFrame
        Daily time series with columns 'date' and 'is_bear_market'
    '''
    
    min_date = bear_markets['start'].min()
    max_date = bear_markets['end'].max()
    
    date_range = pd.date_range(start=min_date, end=max_date, freq='D')
    daily_df = pd.DataFrame({'date': date_range})
    
    daily_df['is_bear_market'] = 0
    
    for _, row in bear_markets.iterrows():
        mask = (daily_df['date'] >= row['start']) & (daily_df['date'] <= row['end'])
        daily_df.loc[mask, 'is_bear_market'] = 1
        
    return daily_df

In [4]:
# Applying function to create dataframe of dates and indication if it is during a bear market
daily_bear_market = create_daily_bear_market_indicator(bear_markets)
daily_bear_market.head(1)

,date,is_bear_market
0,1929-09-07,1


In [5]:
def list_business_days(start_date, end_date):
    us_bd = CustomBusinessDay(calendar=USFederalHolidayCalendar())
    date_range = pd.date_range(start=start_date, end=end_date, freq=us_bd)
    return [date.date() for date in date_range]

start_date = '1990-01-01'
end_date = '2015-12-31'
business_days = pd.DataFrame(list_business_days(start_date, end_date), columns=['date']).astype({'date': 'datetime64[ns]'})
business_days 

,date
0,1990-01-02
1,1990-01-03
2,1990-01-04
3,1990-01-05
4,1990-01-08
...,...
6519,2015-12-24
6520,2015-12-28
6521,2015-12-29
6522,2015-12-30


In [6]:
analysis_period_classified = pd.merge(
    left = business_days,
    right = daily_bear_market,
    left_on = 'date',
    right_on = 'date',
    how = 'inner'
).reset_index()

analysis_period_classified

,index,date,is_bear_market
0,0,1990-01-02,0
1,1,1990-01-03,0
2,2,1990-01-04,0
3,3,1990-01-05,0
4,4,1990-01-08,0
...,...,...,...
6519,6519,2015-12-24,0
6520,6520,2015-12-28,0
6521,6521,2015-12-29,0
6522,6522,2015-12-30,0


In [7]:
operations = pd.read_csv("../distance_results/operations.csv").drop(columns=['Unnamed: 0'])
operations

,Semester,Days,S1,S2,Pair,Return,Converged,Count day
0,0,33,JCI,1436513D,JCI-1436513D,1.000034,True,33
1,0,13,JCI,1436513D,JCI-1436513D,1.000067,True,47
2,0,28,JCI,1436513D,JCI-1436513D,1.000088,True,78
3,0,9,JCI,1436513D,JCI-1436513D,1.000049,True,87
4,0,17,JCI,1436513D,JCI-1436513D,1.000052,True,105
...,...,...,...,...,...,...,...,...
2740,48,8,XEL,WEC,XEL-WEC,1.022778,True,6106
2741,48,7,XEL,WEC,XEL-WEC,1.024433,True,6117
2742,48,10,PNW,WEC,PNW-WEC,1.020942,True,6070
2743,48,5,PNW,WEC,PNW-WEC,1.018191,True,6106


In [8]:
operations = pd.merge(
    left = operations,
    right = analysis_period_classified,
    left_on = 'Count day',
    right_on = 'index',
    how = 'left'
).drop(columns=['index'])

operations

,Semester,Days,S1,S2,Pair,Return,Converged,Count day,date,is_bear_market
0,0,33,JCI,1436513D,JCI-1436513D,1.000034,True,33,1990-02-20,0
1,0,13,JCI,1436513D,JCI-1436513D,1.000067,True,47,1990-03-12,0
2,0,28,JCI,1436513D,JCI-1436513D,1.000088,True,78,1990-04-24,0
3,0,9,JCI,1436513D,JCI-1436513D,1.000049,True,87,1990-05-07,0
4,0,17,JCI,1436513D,JCI-1436513D,1.000052,True,105,1990-06-01,0
...,...,...,...,...,...,...,...,...,...,...
2740,48,8,XEL,WEC,XEL-WEC,1.022778,True,6106,2014-05-02,0
2741,48,7,XEL,WEC,XEL-WEC,1.024433,True,6117,2014-05-19,0
2742,48,10,PNW,WEC,PNW-WEC,1.020942,True,6070,2014-03-13,0
2743,48,5,PNW,WEC,PNW-WEC,1.018191,True,6106,2014-05-02,0


In [9]:
operations['is_bear_market'].value_counts()

is_bear_market
0    2355
1     390
Name: count, dtype: int64